# Exercise: Cross-Validation with Symmetric Pair-Input Data

This exercise consists of two tasks. The first task is compulsory: you will not get the right to take the exam if you fail the first task. The second task optional: you do not have to complete the second task but a successful completion will give you an extra point in the exam.

In both tasks, use the K-nearest neighbors classifier with K=1 and Euclidean distance for learning and the concordance index for evaluation. You are encouraged to re-use your own code from the previous exercises. Use the data files `pairs.data`, `features.data`, and `labels.data` that are available in Moodle. The descriptions of these files are provided in the exercise overview, which is also available in Moodle.

Follow the general exercise guidelines of the course (listed in Moodle). Particularly,

- Describe and implement your solution directly to this Jupyter notebook file.
- Remember to describe your solution in general and add detailed comments to the critical parts of your code.
- Remember to justify your design choices and discuss your results.
- Your report must be easy to follow and your code must be runnable in Jupyter notebook.

Feel free to use markdown cells and code cells as you see appropriate.

Submit the finished work to Moodle before the **deadline Monday 18th of February 2019 at 23:59**. Late submissions will be ignored.

## Cover page

Yue Ma
520790

## Task 1 (compulsory)

**You must successfully complete this task in order to get the right to take the exam.**

1. Implement the modified leave-one-out cross-validation scheme that is described in the lecture notes.

2. Estimate and report the generalisation performance of the K-nearest neighbor classifier in predicting the functional similarity of proteins. Use both the unmodified and the modified leave-one-out cross-validation.

3. Discuss your results. In particular, answer the following questions:
 - Why do the two cross-validation schemes produce notably different estimates?
 - Which scheme is appropriate for estimating the generalisation performance on which types of pairs (A, B, or C) and why?

### Libraries

In [3]:
import pandas as pd
import numpy as np
import pdb
from scipy.stats import zscore
from sklearn.neighbors import KNeighborsClassifier

### load data

In [4]:
features= np.genfromtxt('/Users/mayue/Desktop/TKO-2096/Exe/ExeV/features.data',delimiter=',')
labels=np.genfromtxt('/Users/mayue/Desktop/TKO-2096/Exe/ExeV/labels.data',delimiter=',')
pairs=np.genfromtxt('/Users/mayue/Desktop/TKO-2096/Exe/ExeV/pairs.data',dtype='string',delimiter=',')


### Implement C-index


This part is directly from my previous exercise. It works well and calculate c-index for two given sequences.

In [5]:
def C_index(pred,true_labels):
    """pred and true_labels are sequences which have same length"""
    n=0
    h_sum=0.0
    for i in range(0,len(true_labels)):
        t=true_labels[i]
        p=pred[i]
        for j in range(i+1,len(true_labels)):
            nt=true_labels[j]
            np=pred[j]
            if t!=nt:
                n=n+1
#                 pdb.set_trace()
                if ((p<np)&(t<nt))|((p>np)&(t>nt)):
                    h_sum+=1.0
                else:
                    if p==np:
                        h_sum+=0.5
    return float(h_sum/n)

### Implement unmodified LOO CV

- This class is also from my previous exercise. But for this case I have made some changes.
- The reason why I build it as a class is that `sklearn` has similary design ideas. And I think this capsulation makes the code easier to read and use.
- I did not use my own KNN predictor because I think `sklearn` is more stable and fast. And the previous course email has said that it is acceptable to use `sklearn`'s classifier.
- To use the class, firstly there should `load_data()`method, then use `.predict()`method to save the results into the object. Finally use `evaluate()`method to get the performance of the model.
- The model selection part is removed because it is unnecessary for this exercise.


In [6]:
class Loo_CV():
    def __init__(self):
         pass
    
    def load_data(self,features,labels,pairs):
        """load data set"""
        self.features=features
        self.labels=labels
        self.pairs=pairs
        
    def predict_KNN(self,num_neighbors):
        n=self.features.shape[0] #number of pairs
        results=[]
        for i in range(0,n):#i is the index of test pair
            X_test=self.features[i]
            X_test=X_test.reshape(1,-1)#reshape the test pair to make it works with the predictor (because there is only one sample)
            y_test=self.features[i]
            
            train_indexes=list(set(range(0,n))-set([i]))
#             pdb.set_trace()
            X_train=self.features[train_indexes]
            y_train=self.labels[train_indexes]
            
            predictor=KNeighborsClassifier(num_neighbors)
            predictor.fit(X_train,y_train)
            pred=predictor.predict(X_test)[0]
            
            results.append(pred)
        self.results=results
        
    def evaluate(self,scorer):
        """use given scorer to evaluate the performance of the model"""

        self.score=scorer(self.results,labels)
        return self.score

### Implement modified LOO CV

- the main part of this CV class is similar to the above one. The main difference is lines which are used to filter those ones who share members with the test pair.

In [7]:
class Modified_Loo_CV():
    """Cross-Validation which removes shared objects"""
    
    def __init__(self):
         pass
    
    def load_data(self,features,labels,pairs):
        """load data set"""
        self.features=features
        self.labels=labels
        self.pairs=pairs
        
    
    def predict_KNN(self,num_neighbors):
        n=self.features.shape[0] #number of pairs
        results=[]
        for i in range(0,n):#i is the index of test pair
            test_a=self.pairs[i][0]
            test_b=self.pairs[i][1]
            X_test=self.features[i]
            X_test=X_test.reshape(1,-1)#reshape the test pair to make it works with the predictor (because there is only one sample)
            y_test=self.features[i]
            
            train_indexes=[]
            for j in range(0,n): #Traverse pairs again to find correct tarin set
                if (self.pairs[j][0]!=test_a)&(self.pairs[j][0]!=test_b)&(self.pairs[j][1]!=test_a)&(self.pairs[j][1]!=test_b):
                    train_indexes.append(j) #if j th pairs doesn't have shared obejects with the test pair, then add into the indexes list
#                     pdb.set_trace()
            X_train=self.features[train_indexes]
            y_train=self.labels[train_indexes]
            
            predictor=KNeighborsClassifier(num_neighbors)
            predictor.fit(X_train,y_train)
            pred=predictor.predict(X_test)[0]
            
            results.append(pred)
        self.results=results
        
    def evaluate(self,scorer):
        """use given scorer to evaluate the performance of the model"""

        self.score=scorer(self.results,labels)
        return self.score

            
     

### Results

#### Unmodified LOO CV

In [8]:
loo=Loo_CV()
loo.load_data(features,labels,pairs)
loo.predict_KNN(1)
loo.evaluate(C_index)

0.7617702448210922

The performance of unmodifed CV is  `0.7617702448210922`.

#### Modified LOO CV

In [54]:
mloo=Modified_Loo_CV()
mloo.load_data(features,labels,pairs)
mloo.predict_KNN(1)
mloo.evaluate(C_index)

0.6313559322033898

The performance of modifed CV is  `0.6313559322033898`.

### Discussion

- the two cross-validation schemes produce notably different estimates. Unmodified CV perform notably better than modified CV.
    - This is because in unmodified CV, it uses pairs which "leak" the information of test pairs to train set, which make the model optimistic for case B and Case C (so looks like perform better).
    - On the other hand, modified CV just use ones who have no dependencies with the test pair to train the model, which makes it works well with case C, but it does not involve Case A and B(so performance is worse beacuse it is more "strict").
    
- The modified CV is appropriate for estimating Type C pairs because it is trained under Case C and designed for predicting ones which do not have dependencies with the training set.
- The unmodified CV is good for estimating Type A mostly because it considers the strong dependencies between known ones and unknown ones. For type B it is a bit optimistic, and for type C it is over-optimistic.

## Task 2 (optional)

**Successfully completing this task will give you an extra point in the exam.**

1. Design a leave-one-out cross-validation scheme that is appropriate for estimating the generalisation performance on the type of pairs for which the two aforementioned schemes are not appropriate.

2. Explain why your cross-validation scheme is appropriate.

3. Implement your cross-validation scheme. Estimate and report the generalisation performance as in the first task.

4. Discuss your results. In particular, compare the results to those you obtained in the first task and give reasons for any similarities or differences you observe.

- the two aforementioned schemes are not appropriate are not appropriate for type B pairs.
- For unmodified CV, it works well with Type A. 
- For modifeied CV, it works well with Type C.

---
- So task here is to make the CV works with type B pairs, where unknown ones have only one shared member with known ones.

### Implement LOO CV for type B

- Similary to modefied CV, this modified CV's main difference is that how it choose the training set.
- I just adopted the most naive ways to judge which ones are needed. To make the code eaiser to read, I capsulate this part into a function, which is `judge_typeB()`.

In [56]:
class B_Loo_CV():
    
    
    def __init__(self):
         pass
    
    def load_data(self,features,labels,pairs):
        """load data set"""
        self.features=features
        self.labels=labels
        self.pairs=pairs
    
    def judge_typeB(pair1,pair2):
        """to judge whether one pair has ONLY one shared member with another pair"""
        cond1=(pair1[0]!=pair2[1])&(pair2[0]!=pair2[1])#the members in one pair can't be the same ones
        #==========#
        #the four situation where pair1 has only one shared member with pair2
        cond2=(pair1[0]==pair2[0])&(pair1[1]!=pair2[1]) 
        cond3=(pair1[0]==pair2[1])&(pair1[1]!=pair2[0])
        cond4=(pair1[1]==pair2[0])&(pair1[0]!=pair2[1])
        cond5=(pair1[1]==pair2[1])&(pair1[0]!=pair2[0])
        return (cond1&(cond2|cond3|cond4|cond5))
    
    def predict_KNN(self,num_neighbors):
        n=self.features.shape[0] #number of pairs
        results=[]
        for i in range(0,n):#i is the index of test pair
            test_a=self.pairs[i][0]
            test_b=self.pairs[i][1]
            X_test=self.features[i]
            X_test=X_test.reshape(1,-1)#reshape the test pair to make it works with the predictor (because there is only one sample)
            y_test=self.features[i]
            
            train_indexes=[]
            for j in range(0,n): #Traverse pairs again to find correct tarin set
                if judge_typeB(self.pairs[j],[test_a,test_b]):
                    train_indexes.append(j) #if j th pairs doesn't have shared obejects with the test pair, then add into the indexes list
#                     pdb.set_trace()
            X_train=self.features[train_indexes]
            y_train=self.labels[train_indexes]
            
            predictor=KNeighborsClassifier(num_neighbors)
            predictor.fit(X_train,y_train)
            pred=predictor.predict(X_test)[0]
            
            results.append(pred)
        self.results=results
        
    def evaluate(self,scorer):
        """use given scorer to evaluate the performance of the model"""

        self.score=scorer(self.results,labels)
        return self.score

#### Results and Discussion

- The result of this modified CV is `0.7448210922787194`, a bit lower than that of unmodified CV, but higher than modified CV.
- The reason why its performance is lower than unmodified CV is because it only cares about case B where the known pairs just "leak" part of unknown data's information. This is to say, it doesn't "know" so much as unmodified LOO CV.
- Because type B pairs has more information for the model to learn than type C, so it has better performance than previous modified CV.

- This kind of CV uses pairs which share ONLY one member with unknown data to train, which means it has "got used to" the dependencies of type B pairs. This is the reason why it is appropriate for this type.

In [58]:
bloo=B_Loo_CV()
bloo.load_data(features,labels,pairs)
bloo.predict_KNN(1)
bloo.evaluate(C_index)

0.7448210922787194